In [1]:
import requests
import json
import pandas as pd
import re
import spacy
import numpy as np
import string
import stanza
#import ete4
from nltk.translate import Alignment, AlignedSent, IBMModel1
from nltk.corpus import comtrans
from nltk.metrics import precision
from nltk.metrics import recall
from nltk.translate import alignment_error_rate
from simalign import SentenceAligner
from tqdm import tqdm
import pickle

C:\Users\josep\miniforge3\envs\study\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_url='https://raw.githubusercontent.com/UgaritAlignment/manual-alignments/main/dsh_files/'

In [3]:
test_url='https://raw.githubusercontent.com/UgaritAlignment/manual-alignments/main/dsh_files/1000.json'

In [4]:
text=requests.get(test_url).text

In [5]:
y = json.loads(text)

In [6]:
l1 = y['aligned-text']['language'][0]['@xml:lang']
l2 = y['aligned-text']['language'][1]['@xml:lang']

In [7]:
print(l1,l2)

grc eng


In [8]:
y['aligned-text']['sentence']['wds'][0]['w'][0]['text']

'Αἰσχίνης'

In [9]:
y['aligned-text']['sentence']['wds'][0]['w'][0]['@n']

'1-1'

In [10]:
y['aligned-text']['sentence']['wds'][0]['w'][2]['refs']['@nrefs']

'1-3 1-4'

In [11]:
alignment={}
l1_toks=[]
l2_toks=[]
for record in y['aligned-text']['sentence']['wds'][0]['w']:
    alignment[record['@n']]=record['refs']['@nrefs'].split(' ')
    l1_toks.append(record['text'])
for record in y['aligned-text']['sentence']['wds'][1]['w']:
    l2_toks.append(record['text'])

In [12]:
lang_pairs=set()
for i in tqdm(range(1000,2037)):
    url = base_url+str(i)+".json"
    text=requests.get(test_url).text
    y = json.loads(text)
    l1 = y['aligned-text']['language'][0]['@xml:lang']
    l2 = y['aligned-text']['language'][1]['@xml:lang']
    lang_pairs.add(l1)

  2%|█▊                                                                              | 23/1037 [00:14<10:29,  1.61it/s]


KeyboardInterrupt: 

In [38]:
l1_toks

['Αἰσχίνης',
 'δέ',
 'φησι',
 'καὶ',
 'Λυσικλέα',
 'τὸν',
 'προβατοκάπηλον',
 'ἐξ',
 'ἀγεννοῦς',
 'καὶ',
 'ταπεινοῦ',
 'τὴν',
 'φύσιν',
 'Ἀθηναίων',
 'γενέσθαι',
 'πρῶτον',
 'Ἀσπασίᾳ',
 'συνόντα',
 'μετὰ',
 'τὴν',
 'Περικλέους',
 'τελευτήν',
 '.']

In [41]:
l2_toks

['And',
 'Aeschines',
 'says',
 'that',
 'Lysicles',
 'the',
 'sheepdealer',
 'a',
 'man',
 'of',
 'low',
 'birth',
 'and',
 'nature',
 'came',
 'to',
 'be',
 'the',
 'first',
 'man',
 'at',
 'Athens',
 'by',
 'living',
 'with',
 'Aspasia',
 'after',
 'the',
 'death',
 'of',
 'Pericles',
 '.']

In [ ]:
for token in y['aligned-text']['sentence']['wds'][0]['w']

In [13]:
alignment

{'1-1': ['1-2'],
 '1-2': ['1-1'],
 '1-3': ['1-3', '1-4'],
 '1-4': [''],
 '1-5': ['1-5'],
 '1-6': ['1-6'],
 '1-7': ['1-7'],
 '1-8': ['1-10'],
 '1-9': ['1-11', '1-12'],
 '1-10': ['1-13'],
 '1-11': [''],
 '1-12': [''],
 '1-13': ['1-14'],
 '1-14': ['1-21', '1-22'],
 '1-15': ['1-15', '1-16', '1-17'],
 '1-16': ['1-19'],
 '1-17': ['1-26'],
 '1-18': ['1-23', '1-24', '1-25'],
 '1-19': ['1-27'],
 '1-20': ['1-28'],
 '1-21': ['1-30', '1-31'],
 '1-22': ['1-29'],
 '1-23': ['1-32']}

In [14]:
align_list=[]
for token_idx in range(len(alignment)):
    try:
        aligned_tokens = alignment['1-'+str(token_idx+1)]
    except:
        aligned_tokens = []
    for aligned_token in aligned_tokens:
        if aligned_token != '':
            align_list.append((token_idx,int(aligned_token.split('1-')[1])-1))
aligned_line = AlignedSent(l1_toks,l2_toks,Alignment(align_list))

In [15]:
aligned_line.alignment

Alignment([(0, 1), (1, 0), (2, 2), (2, 3), (4, 4), (5, 5), (6, 6), (7, 9), (8, 10), (8, 11), (9, 12), (12, 13), (13, 20), (13, 21), (14, 14), (14, 15), (14, 16), (15, 18), (16, 25), (17, 22), (17, 23), (17, 24), (18, 26), (19, 27), (20, 29), (20, 30), (21, 28), (22, 31)])

In [16]:
auto_aligned_lines={}
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")
auto_aligned_line = myaligner.get_word_aligns(l1_toks, l2_toks)

2024-04-08 19:53:17,690 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [17]:
alignment_error_rate(aligned_line.alignment,Alignment(auto_aligned_line['mwmf']))

0.6

In [64]:
doc_scores={}

myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")
for i in tqdm(range(1000,2037)):
    url = base_url+str(i)+".json"
    text=requests.get(test_url).text
    y = json.loads(text)

    l1 = y['aligned-text']['language'][0]['@xml:lang']
    l2 = y['aligned-text']['language'][1]['@xml:lang']

    langs = [l1,l2]

    alignment={}
    l1_toks=[]
    l2_toks=[]
    for record in y['aligned-text']['sentence']['wds'][0]['w']:
        alignment[record['@n']]=record['refs']['@nrefs'].split(' ')
        l1_toks.append(record['text'])
    for record in y['aligned-text']['sentence']['wds'][1]['w']:
        l2_toks.append(record['text'])

    align_list=[]
    for token_idx in range(len(alignment)):
        try:
            aligned_tokens = alignment['1-'+str(token_idx+1)]
        except:
            aligned_tokens = []
        for aligned_token in aligned_tokens:
            if aligned_token != '':
                align_list.append((token_idx,int(aligned_token.split('1-')[1])-1))
    aligned_line = AlignedSent(l1_toks,l2_toks,Alignment(align_list))

    auto_aligned_line = myaligner.get_word_aligns(l1_toks, l2_toks)
    

    precision_score={'match':precision(aligned_line.alignment,Alignment(auto_aligned_line['mwmf'])),
               'argmax':precision(aligned_line.alignment,Alignment(auto_aligned_line['inter'])),
               'itermax':precision(aligned_line.alignment,Alignment(auto_aligned_line['itermax']))}
    recall_score={'match':recall(aligned_line.alignment,Alignment(auto_aligned_line['mwmf'])),
               'argmax':recall(aligned_line.alignment,Alignment(auto_aligned_line['inter'])),
               'itermax':recall(aligned_line.alignment,Alignment(auto_aligned_line['itermax']))}
    AER_score={'match':alignment_error_rate(aligned_line.alignment,Alignment(auto_aligned_line['mwmf'])),
               'argmax':alignment_error_rate(aligned_line.alignment,Alignment(auto_aligned_line['inter'])),
               'itermax':alignment_error_rate(aligned_line.alignment,Alignment(auto_aligned_line['itermax']))}

    doc_scores[i]=[langs,precision_score,recall_score,AER_score]

with open('ugarit_dictionary_1.pkl', 'wb') as f:
    pickle.dump(doc_scores, f)

2024-04-03 23:21:20,676 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
100%|███████████████████████████████████████████████████████████████████████████| 1037/1037 [10:07:40<00:00, 35.16s/it]


In [69]:
for value in doc_scores.values():
    if value[0]!=['grc', 'eng']:
        print(value)

In [18]:
import pickle

with open('ugarit_dictionary.pkl', 'rb') as handle:
    doc_scores = pickle.load(handle)

In [23]:
doc_scores[1000][2]

{'match': 0.42857142857142855, 'argmax': 0.25, 'itermax': 0.32142857142857145}

In [ ]:
for line in doc_scores['key']:
    